In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, PReLU,BatchNormalization, Flatten, UpSampling2D, LeakyReLU, Dense, Input, add
from tensorflow.keras.applications import VGG19
from tqdm import tqdm
from PIL import Image
import random


In [3]:
lr_images_path = '../data/lr_images'
hr_images_path = '../data/hr_images'

In [4]:
lr_filenames = sorted(os.listdir(lr_images_path))[:]
hr_filenames = sorted(os.listdir(hr_images_path))[:]

# Verify that the filenames match
if lr_filenames != hr_filenames:
    raise ValueError("Filenames in lr_images and hr_images do not match!")

In [5]:
lr_images = []
hr_images = []


for img_name in lr_filenames:
    lr_img = Image.open(os.path.join(lr_images_path, img_name))
    hr_img = Image.open(os.path.join(hr_images_path, img_name))
    
    # Convert grayscale images to RGB format
    if lr_img.mode != 'RGB':
        lr_img = lr_img.convert('RGB')
        
    if hr_img.mode != 'RGB':
        hr_img = hr_img.convert('RGB')
    
    lr_img = np.asarray(lr_img)/255.0
    hr_img = np.asarray(hr_img)/255.0
    
    lr_images.append(lr_img)
    hr_images.append(hr_img)

In [6]:
lr_images_array = np.stack(lr_images)
hr_images_array = np.stack(hr_images)

print("LR Images Array Shape:", lr_images_array.shape)
print("HR Images Array Shape:", hr_images_array.shape)

LR Images Array Shape: (15129, 64, 64, 3)
HR Images Array Shape: (15129, 128, 128, 3)


In [7]:
# Split the data into training, validation, and test sets
random_seed = 42 

train_lr_images, val_test_lr_images, train_hr_images, val_test_hr_images = train_test_split(
    lr_images_array, hr_images_array, test_size=0.3, random_state=random_seed
)

val_lr_images, test_lr_images, val_hr_images, test_hr_images = train_test_split(
    val_test_lr_images, val_test_hr_images, test_size=0.5, random_state=random_seed
)

In [7]:
#Define blocks to build the generator
def res_block(ip):
    
    res_model = Conv2D(64, (3,3), padding = "same")(ip)
    res_model = BatchNormalization(momentum = 0.5)(res_model)
    res_model = PReLU(shared_axes = [1,2])(res_model)
    
    res_model = Conv2D(64, (3,3), padding = "same")(res_model)
    res_model = BatchNormalization(momentum = 0.5)(res_model)
    
    return add([ip,res_model])

def upscale_block(ip):
    
    up_model = Conv2D(256, (3,3), padding="same")(ip)
    up_model = UpSampling2D(size = 2)(up_model)
    up_model = PReLU(shared_axes=[1,2])(up_model)
    
    return up_model


In [8]:
#Generator model
def create_gen(gen_ip, num_res_block):
    
    layers = Conv2D(64, (9,9), padding="same")(gen_ip)
    layers = PReLU(shared_axes=[1,2])(layers)

    temp = layers

    for i in range(num_res_block):
        layers = res_block(layers)

    layers = Conv2D(64, (3,3), padding="same")(layers)
    layers = BatchNormalization(momentum=0.5)(layers)
    layers = add([layers,temp])

    layers = upscale_block(layers)
    
    op = Conv2D(3, (9,9), padding="same")(layers)

    return Model(inputs=gen_ip, outputs=op)

In [9]:
#Descriminator block that will be used to construct the discriminator
def discriminator_block(ip, filters, strides=1, bn=True):
    
    disc_model = Conv2D(filters, (3,3), strides = strides, padding="same")(ip)
    
    if bn:
        disc_model = BatchNormalization( momentum=0.8 )(disc_model)
    
    disc_model = LeakyReLU( alpha=0.2 )(disc_model)
    
    return disc_model

In [10]:
#Descriminartor, as described in the original paper
def create_disc(disc_ip):

    df = 16
    
    d1 = discriminator_block(disc_ip, df, bn=False)
    d2 = discriminator_block(d1, df, strides=2)
    d3 = discriminator_block(d2, df*2)
    d4 = discriminator_block(d3, df*2, strides=2)
    d5 = discriminator_block(d4, df*4)
    d6 = discriminator_block(d5, df*4, strides=2)

    d8_5 = Flatten()(d6)
    d9 = Dense(df*16)(d8_5)
    d10 = LeakyReLU(alpha=0.2)(d9)
    validity = Dense(1, activation='sigmoid')(d10)

    return Model(disc_ip, validity)

In [11]:
def build_vgg(hr_shape):
    
    vgg = VGG19(weights="imagenet",include_top=False, input_shape=hr_shape)
    
    return Model(inputs=vgg.inputs, outputs=vgg.layers[10].output)

In [12]:
#Combined model
def create_comb(gen_model, disc_model, vgg, lr_ip, hr_ip):
    gen_img = gen_model(lr_ip)
    
    gen_features = vgg(gen_img)
    
    disc_model.trainable = False
    validity = disc_model(gen_img)
    
    return Model(inputs=[lr_ip, hr_ip], outputs=[validity, gen_features])


In [13]:
lr_shape = (lr_images_array.shape[1], lr_images_array.shape[2], lr_images_array.shape[3])
hr_shape = (hr_images_array.shape[1], hr_images_array.shape[2], hr_images_array.shape[3])

In [14]:
lr_images_array.shape

(15129, 64, 64, 3)

In [15]:
lr_ip = Input(shape=lr_shape)
hr_ip = Input(shape=hr_shape)

generator = create_gen(lr_ip, num_res_block = 16)
generator.summary()

Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-09-19 17:54:54.317483: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-19 17:54:54.318440: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 64)   15616       ['input_1[0][0]']                
                                                                                                  
 p_re_lu (PReLU)                (None, 64, 64, 64)   64          ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 64, 64, 64)   36928       ['p_re_lu[0][0]']                
                                                                                              

In [16]:
discriminator = create_disc(hr_ip)
discriminator.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
discriminator.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d_36 (Conv2D)          (None, 128, 128, 16)      448       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 128, 128, 16)      0         
                                                                 
 conv2d_37 (Conv2D)          (None, 64, 64, 16)        2320      
                                                                 
 batch_normalization_33 (Bat  (None, 64, 64, 16)       64        
 chNormalization)                                                
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 64, 64, 16)        0         
                                                           

In [17]:
vgg = build_vgg((128,128,3))
vgg.trainable = False
print(vgg.summary())

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0   

In [18]:
srgan_model = create_comb(generator, discriminator, vgg, lr_ip, hr_ip)
srgan_model.compile(loss=["binary_crossentropy", "mse"], loss_weights=[1e-3, 1], optimizer="adam")
srgan_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 model (Functional)             (None, 128, 128, 3)  1453955     ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 model_1 (Functional)           (None, 1)            4267729     ['model[0][0]']            

In [19]:
batch_size = 16  
train_lr_batches = []
train_hr_batches = []
for it in range(int(hr_images_array.shape[0] / batch_size)):
    start_idx = it * batch_size
    end_idx = start_idx + batch_size
    train_hr_batches.append(hr_images_array[start_idx:end_idx])
    train_lr_batches.append(lr_images_array[start_idx:end_idx])
    

In [20]:
epochs = 20
for e in range(epochs):

    print(f"Epoch {e+1} starting")
    
    fake_label = np.zeros((batch_size, 1)) # Assign a label of 0 to all fake (generated images)
    real_label = np.ones((batch_size,1)) # Assign a label of 1 to all real images.
    
    #Create empty lists to populate gen and disc losses. 
    g_losses = []
    d_losses = []
    
    #Enumerate training over batches. 
    for b in tqdm(range(len(train_hr_batches))):
        lr_imgs = train_lr_batches[b] #Fetch a batch of LR images for training
        hr_imgs = train_hr_batches[b] #Fetch a batch of HR images for training
        
        fake_imgs = generator.predict_on_batch(lr_imgs) #Fake images
        
        #First, train the discriminator on fake and real HR images. 
        discriminator.trainable = True
        d_loss_gen = discriminator.train_on_batch(fake_imgs, fake_label)
        d_loss_real = discriminator.train_on_batch(hr_imgs, real_label)
        
        #Now, train the generator by fixing discriminator as non-trainable
        discriminator.trainable = False
        
        #Average the discriminator loss, just for reporting purposes. 
        d_loss = 0.5 * np.add(d_loss_gen, d_loss_real) 
        
        #Extract VGG features, to be used towards calculating loss
        image_features = vgg.predict(hr_imgs)
     
        #Train the generator via GAN. 
        #Remember that we have 2 losses, adversarial loss and content (VGG) loss
        g_loss, _, _ = srgan_model.train_on_batch([lr_imgs, hr_imgs], [real_label, image_features])
        
        #Save losses to a list so we can average and report. 
        d_losses.append(d_loss)
        g_losses.append(g_loss)
        
    #Convert the list of losses to an array to make it easy to average    
    g_losses = np.array(g_losses)
    d_losses = np.array(d_losses)
    
    #Calculate the average losses for generator and discriminator
    g_loss = np.sum(g_losses, axis=0) / len(g_losses)
    d_loss = np.sum(d_losses, axis=0) / len(d_losses)
    
    #Report the progress during training. 
    print("epoch:", e+1 ,"g_loss:", g_loss, "d_loss:", d_loss)

    with open('training_logs.txt', 'a+') as log_file:
        log_file.write(f"Epoch: {e+1}, Generative Loss: {g_loss}, Discriminator Loss: {d_loss}")
     

    if (e+1) % 2 == 0: #Change the frequency for model saving, if needed
        #Save the generator after every n epochs (Usually 10 epochs)
        generator.save("./generator_ckpts/gen_e_"+ str(e+1) +".h5")

Epoch 1 starting


  0%|          | 0/945 [00:00<?, ?it/s]2023-09-19 17:54:57.094925: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-09-19 17:54:57.096909: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-19 17:54:58.311468: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 114ms/step


2023-09-19 17:54:59.076290: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-09-19 17:55:00.479086: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
  0%|          | 1/945 [00:09<2:27:09,  9.35s/it]

1/1 [==============================] - 0s 55ms/step


...

1/1 [==============================] - 0s 63ms/step


 10%|▉         | 91/945 [06:32<57:36,  4.05s/it]

1/1 [==============================] - 0s 84ms/step


 10%|▉         | 91/945 [06:37<1:02:09,  4.37s/it]


KeyboardInterrupt: 

In [8]:
generator = load_model('results/generator_ckpts/gen_e_6.h5')

2023-09-20 11:12:56.210861: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-20 11:12:56.212194: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [9]:
preds = generator.predict(test_lr_images)

2023-09-20 11:12:59.981140: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-09-20 11:13:00.224206: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


71/71 [==============================] - 111s 2s/step


In [1]:
def calculate_psnr(img1, img2):
    """
    Calculate PSNR (Peak Signal-to-Noise Ratio) for a pair of images.

    Parameters:
    img1 (numpy.ndarray): Ground truth image.
    img2 (numpy.ndarray): Predicted or reconstructed image.

    Returns:
    float: PSNR value.
    """
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = np.max(img1)
    psnr = 10 * np.log10((max_pixel ** 2) / mse)
    return psnr


In [14]:
psnr = []
for preds, original in tqdm(zip(preds, test_hr_images)):
    psnr.append(calculate_psnr(preds, original))

avg_psnr = np.mean(psnr)

print(avg_psnr)

2270it [00:03, 583.98it/s]

30.29189512544407


: 